### Dataset - Load Data

Start by importing the simulator data from the training_data directory.

In [1]:
import csv
import os
import numpy as np

def read_csv(filepath, num_features=7, delimiter=';'):
    data_array = np.array(np.zeros(shape=num_features), ndmin=2)    
    with open(filepath, newline='') as csvfile:
        annotations_reader = csv.reader(csvfile, delimiter=delimiter, quotechar='|')
        for row in annotations_reader:
            data_array = np.vstack((data_array, np.array(row, ndmin=2)))
    return data_array[2:]

In [2]:
DRIVING_LOG_CSV = 'driving_log.csv'
annotations = read_csv(os.path.join('merged_data', DRIVING_LOG_CSV), delimiter=',')
center_image_paths, center_measurements = annotations[:,0], annotations[:,3]
left_image_paths, right_image_paths = annotations[:,1], annotations[:,2]

print('center_image_paths shape', center_image_paths.shape)
print('left_image_paths shape', left_image_paths.shape)
print('right_image_paths shape', right_image_paths.shape)
print('measurements shape', center_measurements.shape)

center_image_paths shape (255,)
left_image_paths shape (255,)
right_image_paths shape (255,)
measurements shape (255,)


In [3]:
import cv2
TRAINING_DATA_DIR = 'merged_data'

def load_images(image_paths):
    images = np.array(np.zeros(shape=(1, 160, 320, 3)), ndmin=4)
    for path in image_paths:
        full_path = os.path.join(TRAINING_DATA_DIR,path.strip())
        if os.path.exists(full_path):
            image = np.array(cv2.imread(full_path), ndmin=4)
            images = np.vstack((images, image))
        else:
            print('Path:',full_path,"does not exist")
    return images[1:]

left_images = load_images(left_image_paths)
right_images = load_images(right_image_paths)
center_images = load_images(center_image_paths)
print(' center_images.shape', center_images.shape, '\n',
     'right_images.shape', right_images.shape, '\n',
     'left_images.shape', left_images.shape)

 center_images.shape (255, 160, 320, 3) 
 right_images.shape (255, 160, 320, 3) 
 left_images.shape (255, 160, 320, 3)


### Append Left and Right Camera Data

In [9]:
# create adjusted steering measurements for the side camera images
correction = 0.23 # tuned parameter
center_measurements = center_measurements.astype('float32')

left_measurements = center_measurements + correction
right_measurements = center_measurements - correction

X_train = np.vstack((center_images, left_images, right_images))
Y_train = np.hstack((center_measurements, left_measurements, right_measurements))

print('X_train.shape', X_train.shape)
print('Y_train.shape', Y_train.shape)

X_train.shape (765, 160, 320, 3)
Y_train.shape (765,)


### Dataset - Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. Image Augmentation - Image Jittering

In [10]:
# TODO: Shuffle the data
perm = np.arange(X_train.shape[0])
np.random.shuffle(perm)
X_train, Y_train = X_train[perm], Y_train[perm]

In [11]:
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5,0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a=-0.5
    b=0.5
    grayscale_min = 0.
    grayscale_max = 255.
    return a + (((image_data - grayscale_min)*(b-a))/(grayscale_max - grayscale_min))

### Train the Network - Model

In [26]:
import tensorflow as tf
tf.python.control_flow_ops = tf
from keras.models import Graph
from keras.layers.core import Flatten, Dense, Dropout, Activation, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from keras.layers import Cropping2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

#Hyperparameters
batch_size = 128
nb_epoch = 10
ch, row, col = 3, 160, 320  # camera format
top_crop = 65
bottom_crop = 25

model = Graph()
model.add_input(name='input',input_shape=(row,col,ch))
model.add_node(Cropping2D(cropping=((top_crop,bottom_crop), (0,0))), name='crop', input='input')
model.add_node(Lambda(lambda x: normalize_grayscale(x)), name='lambda', input='crop')
model.add_node(Convolution2D(16, 8, 8, init='glorot_uniform',
                             subsample=(4,4),border_mode='same'), name='conv_1', input='lambda')
model.add_node(ELU(), input='conv_1', name='elu_1')
model.add_node(Convolution2D(32, 5, 5, init='glorot_uniform',
                             subsample=(2,2),border_mode='same'), name='conv_2', input='elu_1')
model.add_node(ELU(), input='conv_2', name='elu_2')
model.add_node(Convolution2D(64, 5, 5, init='glorot_uniform',
                             subsample=(2,2),border_mode='same'), name='conv_3', input='elu_2')
model.add_node(Flatten(),name='flatten',input='conv_3')
model.add_node(Dropout(0.2),input='flatten',name='dropout_1')
model.add_node(ELU(), input='dropout_1', name='elu_3')
model.add_node(Dense(512),input='elu_3', name='fc_1')
model.add_node(Dropout(0.5),input='fc_1',name='dropout_2')
model.add_node(ELU(), input='dropout_2', name='elu_4')
model.add_node(Dense(1, activation='tanh'),input='elu_4', name='fc_final')
model.add_output(name='output',input='fc_final')

adam = Adam(lr=0.0007, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss={'output':'mse'},
             optimizer=adam,
             metrics={'output':'msle'})
print(model.summary())

callback1 = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                            verbose=0, save_best_only=False, mode='auto')
model.fit({'input':X_train,'output':Y_train}, batch_size=batch_size,
         nb_epoch=nb_epoch, validation_split=0.222, shuffle=True,callbacks=[callback1])
model.save('model.h5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
crop (Cropping2D)                (None, 70, 320, 3)    0           input[0][0]                      
____________________________________________________________________________________________________
lambda (Lambda)                  (None, 70, 320, 3)    0           crop[0][0]                       
____________________________________________________________________________________________________
conv_1 (Convolution2D)           (None, 18, 80, 16)    3088        lambda[0][0]                     
___________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
#Alternate Model

import tensorflow as tf
tf.python.control_flow_ops = tf
from keras.layers import Cropping2D
from keras.models import Graph, Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize, imsave
import cv2
import sys
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


#Hyperparameters
batch_size = 128
nb_epoch = 10
ch, row, col = 3, 160, 320  # camera format
top_crop = 65
bottom_crop = 25


# Implementing SqueezeNet
model = Graph()
model.add_input(name='input',input_shape=(row,col,ch))
model.add_node(Cropping2D(cropping=((top_crop,bottom_crop), (0,0))), name='crop', input='input')
model.add_node(Lambda(lambda x: normalize_grayscale(x)), name='lambda', input='crop')

#conv_1:
model.add_node(Convolution2D(96, 3, 3, activation='relu', init='glorot_uniform',
                             subsample=(2,2),border_mode='valid'),name='conv_1', input='lambda')

#maxpool_1
model.add_node(MaxPooling2D(pool_size=(3,3),strides=(2,2)),name='maxpool_1', input='conv_1')

#fire module 1
model.add_node(Convolution2D(16, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_squeeze', input='maxpool_1')
model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_expand1', input='fire2_squeeze')
model.add_node(Convolution2D(64, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire2_expand2', input='fire2_squeeze')
model.add_node(Activation("linear"),name='fire_2', inputs=["fire2_expand1","fire2_expand2"], merge_mode="concat", concat_axis=1)

#fire module 2
model.add_node(Convolution2D(16, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_squeeze', input='fire_2')
model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_expand1', input='fire3_squeeze')
model.add_node(Convolution2D(64, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire3_expand2', input='fire3_squeeze')
model.add_node(Activation("linear"),name='fire_3', inputs=["fire3_expand1","fire3_expand2"], merge_mode="concat", concat_axis=1)

# # #fire module  3
# model.add_node(Convolution2D(32, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_squeeze', input='fire_3')
# model.add_node(Convolution2D(128, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_expand1', input='fire4_squeeze')
# model.add_node(Convolution2D(128, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire4_expand2', input='fire4_squeeze')
# model.add_node(Activation("linear"),name='fire_4', inputs=["fire4_expand1","fire4_expand2"], merge_mode="concat", concat_axis=1)

# # maxpool 4
# model.add_node(MaxPooling2D((2,2)),name='maxpool_4', input='fire_4')

# # #fire module  5
# model.add_node(Convolution2D(32, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_squeeze', input='maxpool_4')
# model.add_node(Convolution2D(128, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_expand1', input='fire5_squeeze')
# model.add_node(Convolution2D(128, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire5_expand2', input='fire5_squeeze')
# model.add_node(Activation("linear"),name='fire_5', inputs=["fire5_expand1","fire5_expand2"], merge_mode="concat", concat_axis=1)

# # #fire module 6
# model.add_node(Convolution2D(48, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_squeeze', input='fire_5')
# model.add_node(Convolution2D(192, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_expand1', input='fire6_squeeze')
# model.add_node(Convolution2D(192, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire6_expand2', input='fire6_squeeze')
# model.add_node(Activation("linear"),name='fire_6', inputs=["fire6_expand1","fire6_expand2"], merge_mode="concat", concat_axis=1)

# # #fire module 7
# model.add_node(Convolution2D(48, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_squeeze', input='fire_6')
# model.add_node(Convolution2D(192, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_expand1', input='fire7_squeeze')
# model.add_node(Convolution2D(192, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire7_expand2', input='fire7_squeeze')
# model.add_node(Activation("linear"),name='fire_7', inputs=["fire7_expand1","fire7_expand2"], merge_mode="concat", concat_axis=1)

# # #fire module 8
# model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_squeeze', input='fire_7')
# model.add_node(Convolution2D(256, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_expand1', input='fire8_squeeze')
# model.add_node(Convolution2D(256, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire8_expand2', input='fire8_squeeze')
# model.add_node(Activation("linear"),name='fire_8', inputs=["fire8_expand1","fire8_expand2"], merge_mode="concat", concat_axis=1)

# #maxpool 8
# model.add_node(MaxPooling2D((2,2)),name='maxpool_8', input='fire_8')

# # #fire module 9
# model.add_node(Convolution2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_squeeze', input='maxpool_8')
# model.add_node(Convolution2D(256, 1, 1, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_expand1', input='fire9_squeeze')
# model.add_node(Convolution2D(256, 3, 3, activation='relu', init='glorot_uniform',border_mode='same'),name='fire9_expand2', input='fire9_squeeze')
# model.add_node(Activation("linear"),name='fire_9', inputs=["fire9_expand1","fire9_expand2"], merge_mode="concat", concat_axis=1)

model.add_node(Dropout(0.5),input='fire_3',name='fire9_dropout')

#conv_10
model.add_node(Convolution2D(1, 1, 1, activation='relu', init='glorot_uniform',border_mode='valid'),
               name='conv_10', input='fire9_dropout')

#avgpool_10 - Global Average Pooling
model.add_node(GlobalAveragePooling2D((13,13)),name='avgpool_10', input='conv_10')


model.add_node(Flatten(),name='flatten',input='avgpool_10')
model.add_node(Dense(1, activation='tanh'),input='flatten', name='fc_final')
model.add_output(name='output',input='fc_final')

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer="adam", loss="mse")
print(model.summary())

callback1 = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                            verbose=0, save_best_only=False, mode='auto')
model.fit({'input':X_train,'output':Y_train}, batch_size=batch_size,
         nb_epoch=nb_epoch, validation_split=0.222, shuffle=True,callbacks=[callback1])